<a href="https://colab.research.google.com/github/yokosmina/Taller-Ciencia-/blob/main/TallerDia3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import numpy as np

url ='https://raw.githubusercontent.com/yokosmina/Taller-Ciencia-/main/Volados.csv'
df = pd.read_csv(url)
#df.loc[df['a/s'] == 'a', 'clase'] = 1
#df.loc[df['a/s'] == 's', 'clase'] = 0

df


,Try,H-ind,Weight-ind,H-lan,Thumb-L,Wind,Dist-flr-thm,flr-type,sit-stand,a/s
0,1,180,66,77,3.0,no,64,granito,sentado,a
1,2,180,66,80,3.0,no,64,granito,sentado,s
2,3,180,66,80,3.0,no,64,granito,sentado,a
3,4,180,66,73,3.0,no,64,granito,sentado,s
4,5,180,66,70,3.0,no,64,granito,sentado,s
...,...,...,...,...,...,...,...,...,...,...
95,96,170,89,15,7.2,no,123,tela,sentado,s
96,97,170,89,35,7.2,no,123,piel,sentado,a
97,98,170,89,70,7.2,si,123,adoquin,parado,s
98,99,170,89,35,7.2,si,123,piel,sentado,s


In [39]:
df['class'] = np.where(df['a/s'] == 'a',1,0)

df['wind_no'] = np.where(df['Wind'] == 'no',1,0)
df['wind_si'] = np.where(df['Wind'] == 'si',1,0)

df['flr-type_granito'] = np.where(df['flr-type'] == 'granito',1,0)
df['flr-type_adoquin'] = np.where(df['flr-type'] == 'adoquin',1,0)
df['flr-type_lozeta'] = np.where(df['flr-type'] == 'lozeta',1,0)
df['flr-type_piel'] = np.where(df['flr-type'] == 'piel',1,0)
df['flr-type_concreto'] = np.where(df['flr-type'] == 'concreto',1,0)
df['flr-type_madera'] = np.where(df['flr-type'] == 'madera',1,0)
df['flr-type_marmol'] = np.where(df['flr-type'] == 'marmol',1,0)
df['flr-type_hormigon'] = np.where(df['flr-type'] == 'hormigon',1,0)
df['flr-type_tela'] = np.where(df['flr-type'] == 'tela',1,0)

df['sit-stand-sit'] = np.where(df['sit-stand'] == 'sentado',1,0)
df['sit-stand-stand'] = np.where(df['sit-stand'] == 'parado',1,0)
df

#crear variables dummies para variables de texto 
#por ejemplo wind, habría wind_no y wind_si


,Try,H-ind,Weight-ind,H-lan,Thumb-L,Wind,Dist-flr-thm,flr-type,sit-stand,a/s,class,wind_no,wind_si,flr-type_granito,flr-type_adoquin,flr-type_lozeta,flr-type_piel,flr-type_concreto,flr-type_madera,flr-type_marmol,flr-type_hormigon,flr-type_tela,sit-stand-sit,sit-stand-stand
0,1,180,66,77,3.0,no,64,granito,sentado,a,1,1,0,1,0,0,0,0,0,0,0,0,1,0
1,2,180,66,80,3.0,no,64,granito,sentado,s,0,1,0,1,0,0,0,0,0,0,0,0,1,0
2,3,180,66,80,3.0,no,64,granito,sentado,a,1,1,0,1,0,0,0,0,0,0,0,0,1,0
3,4,180,66,73,3.0,no,64,granito,sentado,s,0,1,0,1,0,0,0,0,0,0,0,0,1,0
4,5,180,66,70,3.0,no,64,granito,sentado,s,0,1,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,170,89,15,7.2,no,123,tela,sentado,s,0,1,0,0,0,0,0,0,0,0,0,1,1,0
96,97,170,89,35,7.2,no,123,piel,sentado,a,1,1,0,0,0,0,1,0,0,0,0,0,1,0
97,98,170,89,70,7.2,si,123,adoquin,parado,s,0,0,1,0,1,0,0,0,0,0,0,0,0,1
98,99,170,89,35,7.2,si,123,piel,sentado,s,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [40]:
df.to_csv('volados_ed.csv')


In [41]:
df = df.drop(columns=['Wind','flr-type','sit-stand'])
df.columns

Index(['Try', 'H-ind', 'Weight-ind', 'H-lan', 'Thumb-L', 'Dist-flr-thm', 'a/s',
       'class', 'wind_no', 'wind_si', 'flr-type_granito', 'flr-type_adoquin',
       'flr-type_lozeta', 'flr-type_piel', 'flr-type_concreto',
       'flr-type_madera', 'flr-type_marmol', 'flr-type_hormigon',
       'flr-type_tela', 'sit-stand-sit', 'sit-stand-stand'],
      dtype='object')

In [42]:
df

,Try,H-ind,Weight-ind,H-lan,Thumb-L,Dist-flr-thm,a/s,class,wind_no,wind_si,flr-type_granito,flr-type_adoquin,flr-type_lozeta,flr-type_piel,flr-type_concreto,flr-type_madera,flr-type_marmol,flr-type_hormigon,flr-type_tela,sit-stand-sit,sit-stand-stand
0,1,180,66,77,3.0,64,a,1,1,0,1,0,0,0,0,0,0,0,0,1,0
1,2,180,66,80,3.0,64,s,0,1,0,1,0,0,0,0,0,0,0,0,1,0
2,3,180,66,80,3.0,64,a,1,1,0,1,0,0,0,0,0,0,0,0,1,0
3,4,180,66,73,3.0,64,s,0,1,0,1,0,0,0,0,0,0,0,0,1,0
4,5,180,66,70,3.0,64,s,0,1,0,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,170,89,15,7.2,123,s,0,1,0,0,0,0,0,0,0,0,0,1,1,0
96,97,170,89,35,7.2,123,a,1,1,0,0,0,0,1,0,0,0,0,0,1,0
97,98,170,89,70,7.2,123,s,0,0,1,0,1,0,0,0,0,0,0,0,0,1
98,99,170,89,35,7.2,123,s,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [53]:
import scipy as sp
import numpy as np
import pandas as pd
import pickle
import os
import sys
from pathlib import Path
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
import math
from scipy import sparse

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

def getEpsilonAnalysis(df,reportType,ClassColumn,ClassValue,sufix):
	appended_data = []
	n = len(df)
	nc = len(df[df[ClassColumn] == ClassValue])
	nnoc = len(df)-nc
	pc = nc/n
	pnoc = 1-pc
	cont=1
	report = df.melt(id_vars=[ClassColumn]).rename(columns={"variable": "Driver", "value": "Value"})
	report = report.dropna()
	report = report.astype('str').groupby(['Driver','Value'])[ClassColumn].value_counts().reset_index(name='counts')
	report['N'] = n
	report['NC'] = nc
	report['N~C'] = nnoc
	report['P(C)'] = pc
	report['P(~C)'] = pnoc
	coun=report[[ClassColumn,'Driver','Value','counts']]
	nxcs = coun[coun[ClassColumn].astype('str')==str(ClassValue)]
	nxnocs = coun[coun[ClassColumn].astype('str') != str(ClassValue)]
	conc = pd.merge(nxcs,nxnocs,how='left',left_on=['Driver','Value'], right_on = ['Driver','Value'])
	report2 = pd.merge(report,conc,how='left',left_on=['Driver','Value'], right_on = ['Driver','Value'])
	report2 = report2.rename(columns={'counts_x':'NX(C)','counts_y':'NX(~C)'})
	report2 = report2.drop(columns=[ClassColumn,'counts',ClassColumn+'_x',ClassColumn+'_y'])
	report2 = report2.drop_duplicates()
	report2['NULLs'] = sum(report2.isna().sum())
	report2['NX(C)'] = report2['NX(C)'].astype('float')
	report2['NX(~C)'] = report2['NX(~C)'].astype('float')
	report2['NX(C)'] = report2['NX(C)'].fillna(1)
	report2['NX(~C)'] = report2['NX(~C)'].fillna(1)
	report2['NXs'] = report2['NX(C)'].astype('float')+report2['NX(~C)'].astype('float')
	report2['P(C|X)'] = report2['NX(C)'].astype('float').fillna(1)/report2['NXs'].astype('float')
	report2['P(X|C)'] = report2['NX(C)'].astype('float').fillna(1)/nc
	report2['P(X|~C)'] = report2['NX(~C)'].astype('float').fillna(1)/nnoc
	report2['Epsilon'] = (report2['NXs'].astype('float')*((report2['NX(C)'].astype('float').fillna(1)/report2['NXs'].astype('float'))-pc))/(((report2['NXs'].astype('float')*pc)*(1-pc))**(1/2))
	report2['Score'] = np.log((report2['NX(C)'].astype('float').fillna(1)/nc)/(report2['NX(~C)'].astype('float').fillna(1)/nnoc))
	report2=report2[['Driver','Value','Epsilon','Score','N','NC','N~C','NX(C)','NX(~C)','P(C)','P(~C)','P(C|X)','P(X|C)','P(X|~C)']]
	report2.sort_values(reportType, inplace=True,ascending=False)
	report2.to_csv('EpsilonReport'+str(sufix)+'.csv')
	return report2

def getGaussianNBModel(trainfeatures,trainlabels,testfeatures,testlabels,datatype,lang,varclass,users,rw):
	if rw=='w':
		scaler = Normalizer()
		model = GaussianNB()
		trainfeatures_scal=scaler.fit_transform(trainfeatures)
		model.fit(trainfeatures_scal,trainlabels)	
		pickle.dump(scaler, open('GaussianNB_'+str(datatype)+'_scaler_'+str(varclass)+'_'+str(lang)+'.sav', 'wb'))
		pickle.dump(model, open('GaussianNB_'+str(datatype)+'_model_'+str(varclass)+'_'+str(lang)+'.sav', 'wb'))
	elif rw=='r':
		scaler = pickle.load(open('Scalers\\GaussianNB_'+str(datatype)+'_scaler_'+str(varclass)+'_'+str(lang)+'.sav', 'rb'))
		model = pickle.load(open('Models\\GaussianNB_'+str(datatype)+'_model_'+str(varclass)+'_'+str(lang)+'.sav', 'rb'))
	testfeatures_scal = scaler.transform(testfeatures)	
	y_pred = model.predict(testfeatures_scal)
	users['predicted']=y_pred
	users.to_csv('GaussianNB_'+str(datatype)+'_model_'+str(varclass)+'_'+str(lang)+'.csv')
	return users
getEpsilonAnalysis(df[[column for column in df.columns if column != 'Try' and column != 'a/s' ]],'Epsilon','class',1,'Hector')

train, test = train_test_split(df, test_size=0.3)
getGaussianNBModel(train[[column for column in df.columns if column != 'Try' and column != 'a/s' and column !='class']],train['class'],test[[column for column in df.columns if column != 'Try' and column != 'a/s' and column !='class']],test['class'],'moneda','ozumbaHector','class',test,'w')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Try,H-ind,Weight-ind,H-lan,Thumb-L,Dist-flr-thm,a/s,class,wind_no,wind_si,flr-type_granito,flr-type_adoquin,flr-type_lozeta,flr-type_piel,flr-type_concreto,flr-type_madera,flr-type_marmol,flr-type_hormigon,flr-type_tela,sit-stand-sit,sit-stand-stand,predicted
74,75,172,70,70,6.0,96,a,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1
82,83,165,87,32,6.0,125,s,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1
27,28,157,53,112,2.8,57,s,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
58,59,161,72,78,5.0,77,a,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0
10,11,180,66,128,3.0,124,s,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
45,46,158,64,116,6.0,80,a,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0
37,38,163,70,194,5.5,130,a,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0
64,65,170,60,130,6.0,115,s,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
0,1,180,66,77,3.0,64,a,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0
92,93,170,89,30,7.2,123,s,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0


,Driver,Value,Epsilon,Score,N,NC,N~C,NX(C),NX(~C),P(C),P(~C),P(C|X),P(X|C),P(X|~C)
145,Thumb-L,5.5,1.085636,0.653142,100,51,49,8.0,4.0,0.51,0.49,0.666667,0.156863,0.081633
54,H-ind,163.0,1.085636,0.653142,100,51,49,8.0,4.0,0.51,0.49,0.666667,0.156863,0.081633
40,Dist-flr-thm,77.0,1.081195,0.876285,100,51,49,5.0,2.0,0.51,0.49,0.714286,0.098039,0.040816
163,Weight-ind,72.0,1.081195,0.876285,100,51,49,5.0,2.0,0.51,0.49,0.714286,0.098039,0.040816
143,Thumb-L,5.0,1.081195,0.876285,100,51,49,5.0,2.0,0.51,0.49,0.714286,0.098039,0.040816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Weight-ind,89.0,-0.810231,-0.396680,100,51,49,7.0,10.0,0.51,0.49,0.411765,0.137255,0.204082
58,H-ind,170.0,-0.914659,-0.376478,100,51,49,10.0,14.0,0.51,0.49,0.416667,0.196078,0.285714
140,Thumb-L,3.0,-1.027867,-0.491990,100,51,49,7.0,11.0,0.51,0.49,0.388889,0.137255,0.224490
122,H-lan,70.0,-1.040208,-1.138618,100,51,49,1.0,3.0,0.51,0.49,0.250000,0.019608,0.061224


In [ ]:
train = df.head(n=int((70*df.shape[0])/100))
train

,Try,H-ind,Weight-ind,H-lan,Thumb-L,Wind,Dist-flr-thm,flr-type,sit-stand,a/s,clase
0,1,180,66,77,3.0,no,64,granito,sentado,a,1.0
1,2,180,66,80,3.0,no,64,granito,sentado,s,0.0
2,3,180,66,80,3.0,no,64,granito,sentado,a,1.0
3,4,180,66,73,3.0,no,64,granito,sentado,s,0.0
4,5,180,66,70,3.0,no,64,granito,sentado,s,0.0
...,...,...,...,...,...,...,...,...,...,...,...
65,66,170,60,137,6.0,no,115,lozeta,parado,s,0.0
66,67,172,70,37,6.0,no,131,madera,parado,s,0.0
67,68,172,70,37,6.0,no,51,tela,parado,a,1.0
68,69,172,70,37,6.0,no,100,tela,sentado,a,1.0


In [ ]:
#Se obtiene el 30% de datos para entrenamiento
test = df.tail(n=int((30*df.shape[0])/100))
test

,Try,H-ind,Weight-ind,H-lan,Thumb-L,Wind,Dist-flr-thm,flr-type,sit-stand,a/s,clase
70,71,160,55,30,3.0,no,55,madera,sentado,s,0.0
71,72,160,55,25,3.0,no,52,tela,sentado,s,0.0
72,73,165,62,15,4.0,no,43,madera,sentado,a,1.0
73,74,172,70,56,6.0,no,95,marmol,sentado,s,0.0
74,75,172,70,70,6.0,no,96,piel,sentado,a,1.0
75,76,165,87,35,6.0,si,150,concreto,parado,a,1.0
76,77,165,87,65,6.0,si,153,lozeta,parado,a,1.0
77,78,165,87,35,6.0,no,102,lozeta,parado,s,0.0
78,79,165,87,35,6.0,no,101,lozeta,sentado,s,0.0
79,80,165,87,21,6.0,no,30,lozeta,sentado,s,0.0


In [ ]:
columns = train.columns
drivers = [column for column in columns if (column != 'a/s' and column != 'clase')]
drivers

['Try',
 'H-ind',
 'Weight-ind',
 'H-lan',
 'Thumb-L',
 'Wind',
 'Dist-flr-thm',
 'flr-type',
 'sit-stand']

In [ ]:
labels = [column for column in columns if (column == 'a/s' or column == 'clase')]
labels

['a/s', 'clase']

In [ ]:
N = train.shape[0]
N

70

In [ ]:
NC = train[train['clase']==1].shape[0]
NC

38

In [ ]:
NNOC = train[train['clase']==0].shape[0]
NNOC

32

['Try', 'H-ind',  'Weight-ind', 'H-lan', 'Thumb-L',  'Wind',
 'Dist-flr-thm',  'flr-type',  'sit-stand']

In [ ]:
NXsTry = train['Try'].value_counts().reset_index(name='NXsTry')
NXsTry = NXsTry.rename(columns={'index':'Try'})

NXsHind = train['H-ind'].value_counts().reset_index(name='NXsHind')
NXsHind = NXsHind.rename(columns={'index':'H-ind'})

NXsWeind = train['Weigtht-ind'].value_counts().reset_index(name='NXsWeind')
NXsWeind = NXsWeind.rename(columns={'index':'Weight-ind'})

NXsHlan = train['H-lan'].value_counts().reset_index(name='NXsHlan')
NXsHlan = NXsHlan.rename(columns={'index':'H-lan'})

NXsThumb = train['Thumb-L'].value_counts().reset_index(name='NXsThumb')
NXsThumb = NXsThumb.rename(columns={'index':'Thumb-L})

NXsWind = train['Wind'].value_counts().reset_index(name='NXsWind')
NXsWind = NXsWind.rename(columns={'index':'Wind'})

NXsDist = train['Dist-flr-thm'].value_counts().reset_index(name='NXsDist')
NXsDist = NXsDist.rename(columns={'index':'Dist-flr-thm'})

NXsFlr = train['flr-type'].value_counts().reset_index(name='NXsFlr')
NXsFlr = NXsFlr.rename(columns={'index':'flr-type'})

NXsPos = train['sit-stand'].value_counts().reset_index(name='NXsPos')
NXsPos = NXsPos.rename(columns={'index':'sit-stand'})